# Manipulação de Dados



Para começar, importamos `torch`. Note que apesar de ser chamado PyTorch, devemos importar `torch` ao invés de `pytorch`

In [1]:
import torch

Um tensor representa uma matriz (possivelmente multidimensional) de valores numéricos

In [2]:
x = torch.arange(12)
x

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

Podemos acessar o formato do tensor
e o número total de elementos

In [3]:
x.shape

torch.Size([12])

In [4]:
x.numel()

12

Mudar o *shape* de um tensor sem alterar
o número de elementos ou seus valores

In [5]:
X = x.reshape(3, 4)
X

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

Podemos criar um tensor representando um tensor com todos os elementos
definido como 0
ou 1

In [6]:
torch.zeros((2, 3, 4))

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [7]:
torch.ones((2, 3, 4))

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

Amostrar aleatoriamente os valores
para cada elemento em um tensor

In [8]:
torch.randn(3, 4)

tensor([[ 0.0984, -0.2808,  0.4674, -0.4605],
        [ 0.4865, -1.3598, -0.3926,  1.8938],
        [ 1.1266,  0.2372, -1.1405, -1.0904]])

Especificar os valores exatos para cada elemento

In [9]:
torch.tensor([[2, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])

tensor([[2, 1, 4, 3],
        [1, 2, 3, 4],
        [4, 3, 2, 1]])

Os operadores aritméticos padrão comuns
(`+`, `-`,` * `,` / `e` ** `)
foram todos transformados em operações elemento a elemento

In [10]:
x = torch.tensor([1.0, 2, 4, 8])
y = torch.tensor([2, 2, 2, 2])
x + y, x - y, x * y, x / y, x ** y

(tensor([ 3.,  4.,  6., 10.]),
 tensor([-1.,  0.,  2.,  6.]),
 tensor([ 2.,  4.,  8., 16.]),
 tensor([0.5000, 1.0000, 2.0000, 4.0000]),
 tensor([ 1.,  4., 16., 64.]))

Mais operações podem ser aplicadas elemento a elemento

In [11]:
torch.exp(x)

tensor([2.7183e+00, 7.3891e+00, 5.4598e+01, 2.9810e+03])

*concatenar* vários tensores juntos

In [12]:
X = torch.arange(12, dtype=torch.float32).reshape((3,4))
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

Construir um tensor binário por meio de *declarações lógicas*

In [13]:
X == Y

tensor([[False,  True, False,  True],
        [False, False, False, False],
        [False, False, False, False]])

Somando todos os elementos no tensor

In [14]:
X.sum()

tensor(66.)

Realizar operações elementar
invocando o mecanismo de *Broadcasting*

In [15]:
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a, b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

In [16]:
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

`[-1]` seleciona o último elemento e `[1: 3]`
seleciona o segundo e o terceiro elementos

In [17]:
X[-1], X[1:3]

(tensor([ 8.,  9., 10., 11.]),
 tensor([[ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]))

Também podemos escrever elementos de uma matriz especificando índices

In [18]:
X[1, 2] = 9
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  9.,  7.],
        [ 8.,  9., 10., 11.]])

Para atribuir a vários elementos o mesmo valor,
simplesmente indexamos todos eles e, em seguida, atribuímos o valor a eles

In [19]:
X[0:2, :] = 12
X

tensor([[12., 12., 12., 12.],
        [12., 12., 12., 12.],
        [ 8.,  9., 10., 11.]])

As operações em execução podem fazer com que uma nova memória seja
alocado aos resultados do host

In [20]:
before = id(Y)
Y = Y + X
id(Y) == before

False

Executar operações no local

In [21]:
Z = torch.zeros_like(Y)
print('id(Z):', id(Z))
Z[:] = X + Y
print('id(Z):', id(Z))

id(Z): 140253234249792
id(Z): 140253234249792


Se o valor de `X` não for reutilizado em cálculos subsequentes,
também podemos usar `X [:] = X + Y` ou` X + = Y`
para reduzir a sobrecarga de memória da operação

In [22]:
before = id(X)
X += Y
id(X) == before

True

Converter para um tensor NumPy

In [23]:
A = X.numpy()
B = torch.tensor(A)
type(A), type(B)

(numpy.ndarray, torch.Tensor)

Converter um tensor de tamanho 1 em um escalar Python

In [24]:
a = torch.tensor([3.5])
a, a.item(), float(a), int(a)

(tensor([3.5000]), 3.5, 3.5, 3)